In [8]:
%pip install google-generativeai
%pip install e2b_code_interpreter


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
#%pip install google-colab
%pip install google-generativeai

import google.generativeai as genai

model = genai.GenerativeModel('gemini-pro')

import os
from dotenv import load_dotenv
load_dotenv()

# TODO: Get your Google API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = os.getenv("E2B_API_KEY")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from e2b_code_interpreter import Sandbox

In [11]:
import base64
from IPython.display import display, HTML
import io

In [12]:
def display_plot(plot_data):
    """Convert plot data to base64 and display it"""
    if isinstance(plot_data, bytes):
        b64_plot = base64.b64encode(plot_data).decode()
        display(HTML(f'<img src="data:image/png;base64,{b64_plot}"/>'))
    else:
        print("Plot data is not in the expected format")

In [ ]:
def code_interpret(e2b_code_interpreter, code):
    try:
        print("Running code interpreter...")
        clean_code = code.replace('```python', '').replace('```', '').strip()
        
        exec = e2b_code_interpreter.run_code(
            clean_code,
            on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
            on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
            timeout=30
        )

        if exec.error:
            print("[Code Interpreter ERROR]", exec.error)
            return None
            
        # Display the plot if it exists in results
        if exec.results and len(exec.results) > 0:
            display_plot(exec.results[0])
            
        return exec.results
    except Exception as e:
        print(f"Error in code interpreter: {str(e)}")
        return None

def chat(e2b_code_interpreter, user_message):
    try:
        print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

        prompt = f"""You are a python data scientist. Write simple, working code for this task.
Important rules:
- Use only basic imports: numpy, pandas, matplotlib.pyplot
- Always start with all imports including BytesIO
- Always end by saving plot to buffer and returning it
- No plt.show()

Here's the EXACT format to follow:
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Your plotting code here

# Save plot to bytes buffer
buf = BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
plot_data = buf.getvalue()
plt.close()

plot_data  # This must be the last line

User request: {user_message}

Return only the Python code, no explanations."""

        response = model.generate_content(prompt)

        if response.text:
            code = response.text.strip()
            if code:
                print("CODE TO RUN")
                print(code)
                return code_interpret(e2b_code_interpreter, code)

        print("No response generated")
        return None
    except Exception as e:
        print(f"Error in chat function: {str(e)}")
        return None

In [ ]:
try:
    code_interpreter = Sandbox(api_key=E2B_API_KEY)
    print("Sandbox initialized") # Added for debugging - TBD remove
    results = chat(
        code_interpreter,
        "Create a simple scatter plot with 10 random points"
    )
    print(f"Results received: {results}") # Added for debugging - TBD remove

    if results:
        print(f"Result type: {type(results[0])}") # Added for debugging - TBD remove
        plot1 = results[0]
        print("Plot data extracted") # Added for debugging - TBD remove
        display_plot(plot1)
        print("Plot generated successfully")
    else:
        print("Failed to generate plot")

except Exception as e:
    print(f"Error during execution: {str(e)}")
finally:
    try:
        code_interpreter.kill()
    except:
        pass

Sandbox initialized

User Message: Create a simple scatter plot with 10 random points
